In [5]:
import numpy as np
import keras
import cv2
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import os
import random
import glob
import sys

### In train/val/test folder we must keep different class video_clip images inside the corresponding class directory.

## ├───test
#### │   ├───Guard [ Guard_img_001.png,.......]
#### │   ├───Kick [ Kick_img_001.png,.....]
#### │   └───Punch [ Punch_img_001.png,......]
## ├───train
#### │   ├───Guard [ Guard_img_001.png]
#### │   ├───Kick [ Kick_img_001.png]
#### │   └───Punch [ Punch_img_001.png]
## └───val
####    ├───Guard [ Guard_img_001.png]
####    ├───Kick [ Kick_img_001.png]
####    └───Punch [ Punch_img_001.png]



In [14]:
def build_feature_extractor(IMG_SIZE = 224):
    feature_extractor = keras.applications.ResNet50V2(
        weights="imagenet",
        include_top=False,
        pooling="max",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()
feature_extractor.summary()

2021-08-26 16:14:42.221321: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-08-26 16:14:42.245480: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-26 16:14:42.245860: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 3080 computeCapability: 8.6
coreClock: 1.815GHz coreCount: 68 deviceMemorySize: 9.78GiB deviceMemoryBandwidth: 707.88GiB/s
2021-08-26 16:14:42.246388: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-08-26 16:14:42.246577: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'lib

Model: "feature_extractor"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf_op_layer_RealDiv (TensorF [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf_op_layer_Sub (TensorFlowO [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50v2 (Functional)      (None, 2048)              23564800  
Total params: 23,564,800
Trainable params: 23,519,360
Non-trainable params: 45,440
_________________________________________________________________


In [26]:


class Video_DataGenerator(keras.utils.Sequence):    
    
    def __init__(self, video_frames_path, size = 224,batch_size=32,clip_len=25,h=224,w=224,ch=3,shuffle=True,label_dict = {"no_damage":0,"damage":1}):
        'Initialization'

        self.batch_size = batch_size

        self.video_frames_path = video_frames_path
        
        self.video_clip_list = glob.glob(self.video_frames_path+'*/*')
        print(self.video_clip_list)

        self.shuffle = shuffle

        self.clip_len = clip_len

        self.label_dict = label_dict
        
        self.h = h
        self.w = w
        self.ch = ch
        self.size = size

        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.video_clip_list) / self.batch_size))



    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        # Find list of IDs
        list_IDs_temp = [self.video_clip_list[k] for k in indexes]

        X = np.empty((self.batch_size,self.clip_len,self.h,self.w,self.ch))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i,vid in enumerate(list_IDs_temp):

            clip_features = []

            f = glob.glob(vid+'/*.jpg')

            for k,im in enumerate(f):

                label = im.split('/')[5]
                frame = cv2.imread(im)
                frame = cv2.resize(frame,(self.size,self.size))
                frame =  cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#                 feature = feature_extractor.predict(frame[None,])
                clip_features.append(frame)
                
                if k == int(self.clip_len - 1) :
                    break
        

            X[i] = np.array(clip_features)
            y[i] = self.label_dict[label]

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.video_clip_list))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)


In [41]:


train_gen = Video_DataGenerator(train_path)

val_gen = Video_DataGenerator(val_path)


# for i in train_gen:
#     print(i[0].shape)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [43]:
import numpy as np
import keras

class Features_DataGenerator(keras.utils.Sequence):    
    
    def __init__(self, path,feature_dim=2048, size = 224,batch_size=32,clip_len=25,shuffle=True,label_dict = {"no_damage":0,"damage":1}):
        'Initialization'

        self.batch_size = batch_size
        self.path = path 
        self.video_clip_list = glob.glob(self.path+'*/*')

        self.shuffle = shuffle

        self.clip_len = clip_len

        self.label_dict = label_dict
        
        self.features_dim  = feature_dim
        
        self.size = size

        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.video_clip_list) / self.batch_size))



    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.video_clip_list[k] for k in indexes]

        X = np.empty((self.batch_size,self.clip_len,self.features_dim))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i,vid in enumerate(list_IDs_temp):

            clip_features = []

            f = glob.glob(vid+'/*.jpg')

            for k,im in enumerate(f):

                label = im.split('/')[5]
                frame = cv2.imread(im)
                frame = cv2.resize(frame,(self.size,self.size))
                frame =  cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                feature = feature_extractor.predict(frame[None,])
                clip_features.append(feature)
                
                if k == int(self.clip_len - 1) :
                    break
        

            X[i] = np.array(clip_features).reshape(self.clip_len,-1)
            y[i] = self.label_dict[label]

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.video_clip_list))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)


In [ ]:
train_gen = Features_DataGenerator(train_path)

for i in train_gen:
    print(i[0].shape)